# Libs

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_colwidth', None)  # Без ограничения ширины столбцов
pd.set_option('display.width', None)         # Без ограничения ширины вывода
pd.set_option('display.max_columns', None)   # Показать все столбцы

# Reading data

In [4]:
candles = pd.read_csv('../data/new/forecast_data/candles.csv')
print('candles', candles.shape)
# candles['begin'] = pd.to_datetime(candles['begin'])

candles_2 = pd.read_csv('../data/new/forecast_data/candles_2.csv')
print('candles_2', candles_2.shape)

news = pd.read_csv('../data/new/forecast_data/news.csv')
print('news', news.shape)

news_2 = pd.read_csv('../data/new/forecast_data/news_2.csv')
print('news_2', news_2.shape)


candles (24197, 7)
candles_2 (1745, 7)
news (25425, 4)
news_2 (2030, 3)


In [7]:
print(candles.ticker.unique())
candles.head(2)

['AFLT' 'ALRS' 'CHMF' 'GAZP' 'GMKN' 'LKOH' 'MAGN' 'MGNT' 'MOEX' 'MTSS'
 'NVTK' 'PHOR' 'PLZL' 'ROSN' 'RUAL' 'SBER' 'SIBN' 'T' 'VTBR']


,open,close,high,low,volume,begin,ticker
0,81.50,81.7,83.20,81.16,29755530,2020-06-19,AFLT
1,81.72,82.1,83.98,80.26,18502950,2020-06-22,AFLT


In [6]:
news.head(2)

,Unnamed: 0,publish_date,title,publication
0,25389,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов,"Тенденции в отрасли. Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов с развитых и развивающихся рынков как на операционном, так и на финансовом уровнях. Можно отметить и комфортную долговую нагрузку, высокую рентабельность, стабильный денежный поток, высокие дивиденды. Капзатраты сектора высокие, однако большая часть приходится на масштабную инвестпрограмму ""Газпрома"", направленную в т.ч. на развитие ключевого проекта ""Сила Сибири"". 2019 год сложился неблагоприятно с точки зрения долларовых цен на нефть и газ, однако компании смогли заработать на девальвации рубля и экономии на налогах за счет измененного демпфирующего механизма. Отдельно отметим сохраняющийся высокий уровень дивидендных выплат в отрасли. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Выручка, млрд долл. Ключевые события. Введение обратного акциза на нефть (демпфирующий механизм) в начале года было сопряжено со сложностями адаптации к рыночным реалиям, однако корректировка позволила компаниям заработать. Переход к новой дивидендной политике – ""Газпром"" (50% от прибыли по МСФО в 2021 г.) и ""ЛУКОЙЛ"" (приоритет дивидендам, а не выкупу своих акций). Мультипликаторы Источник: Bloomberg: ПСБ Аналитика & Стратегия Риски. Основные риски отрасли сопряжены с возможным дальнейшим ухудшением ценовой конъюнктуры и изменениями в сфере налогообложения. Средние значения денежных потоков, млн долл. Источник: Bloomberg: ПСБ Аналитика & Стратегия Рекомендации: ""Газпром"". - Согласно новой дивидендной политике, ""Газпром"" планирует в течение 3 лет довести коэффициент дивидендных выплат до 50%, благодаря чему дивидендная доходность может превысить 10%. - Комфортная долговая нагрузка, которая не будет препятствовать уменьшению дивидендных выплат. - Достаточный для выплат дивидендов акционерам объем FCF и отсутствие дальнейшего наращивания инвестпрограммы. Динамика цен на нефть и газ, долл./мБТЕ Источник: Bloomberg, ПСБ Аналитика & Стратегия"
1,25388,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на внутренний рынок,"Тенденции в отрасли. Ключевые российские сталелитейные компании сохраняют конкурентоспособность на операционном и финансовом уровне относительно аналогов. Высокая рентабельность сочетается с низкой долговой нагрузкой, стабильными денежными потоками и высокой дивидендной доходностью. Прошедший год был неблагоприятным с точки зрения ценовой конъюнктуры, но высокая премия на внутреннем рынке сыграла в плюс, поэтому существенно просел экспорт. Учитывая циклический характер развития отрасли и замедление промышленного производства, финансовые результаты остались на уровне прошлого года или ухудшились, что оказало давление на котировки. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Funds form Operation, млрд долл. Корпоративные события. Из отраслевых событий отметим введение пошлин на импорт г/к проката, а также запрет на экспорт лома. В период до 2023г. рост спроса на металлургическую продукцию должна обеспечить реализация национальных проектов, которые позволят увеличить ежегодный выпуск стали на 4%. Угрозу дивидендам не несут инвестпрограммы компаний, которые остались без изменения. Из корпоративных событий центральной темой года стала сделка по продажа американского подразделения ТМК. Мультипликаторы Источник: Bloomberg Риски. Условия на рынке сырья складываются неплохо, но сырьевой сегмент уже не способен перекрывать по рентабельности стальной. Внешние условия могут негативно повлиять на FCF и скажутся на снижении дивидендных выплат. Средние значения денежных потоков, млн долл. Источник: ММК, НЛМК, ""Северсталь"", Evraz, ПСБ Аналитика&Стратегия Рекомендации: ММК. - Долговая нагрузка самая низкая среди российских металлургов. Объем денежных средств полностью покрывает 

In [38]:
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import yake
from typing import List, Dict, Tuple
import json

# feature enginearing

In [8]:
import re
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import yake
from typing import List, Dict, Tuple
import json

class TickerRAGSystem:
    def __init__(self, ticker_mapping: Dict[str, str]):
        self.ticker_mapping = ticker_mapping
        self.company_knowledge_base = self._build_knowledge_base()
        
        # Инициализация моделей
        try:
            self.nlp = spacy.load("ru_core_news_sm")
        except OSError:
            self.nlp = None
            
        self.sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        self.kw_extractor = yake.KeywordExtractor(lan="ru", n=2, dedupLim=0.8, top=10)
        
        # Векторизация базы знаний
        self._vectorize_knowledge_base()
    
    def _build_knowledge_base(self) -> Dict[str, Dict]:
        """Строит базу знаний о компаниях"""
        knowledge_base = {}
        
        # Базовые описания отраслей (можно расширять)
        industry_descriptions = {
            'нефтегазовая': ['нефть', 'газ', 'скважина', 'месторождение', 'трубопровод', 'нефтедобыча', 'нефтепереработка'],
            'банковская': ['банк', 'кредит', 'вклад', 'ипотека', 'финансы', 'расчетный счет', 'денежные переводы'],
            'металлургическая': ['металл', 'сталь', 'алюминий', 'никель', 'медь', 'руда', 'прокат'],
            'ритейл': ['магазин', 'розница', 'продажи', 'сеть', 'торговля', 'ритейлер'],
            'телекоммуникации': ['связь', 'интернет', 'телефон', 'мобильный', 'телеком', 'сотовый'],
            'транспорт': ['авиаперевозки', 'аэропорт', 'рейс', 'транспорт', 'логистика'],
            'горнодобывающая': ['шахта', 'рудник', 'добыча', 'полезные ископаемые', 'горное дело'],
            'химическая': ['химия', 'удобрения', 'пластмассы', 'химическая продукция'],
            'энергетика': ['электроэнергия', 'энергия', 'электростанция', 'энергосбыт']
        }
        
        for company_name, ticker in self.ticker_mapping.items():
            # Автоматически определяем отрасль по ключевым словам
            industry = self._detect_industry(company_name, industry_descriptions)
            
            knowledge_base[ticker] = {
                'company_name': company_name,
                'ticker': ticker,
                'industry': industry,
                'keywords': self._generate_company_keywords(company_name, industry, industry_descriptions),
                'aliases': self._generate_aliases(company_name)
            }
        
        return knowledge_base
    
    def _detect_industry(self, company_name: str, industry_descriptions: Dict) -> str:
        """Автоматически определяет отрасль компании"""
        company_lower = company_name.lower()
        
        for industry, keywords in industry_descriptions.items():
            if any(keyword in company_lower for keyword in keywords):
                return industry
        
        return 'другая'
    
    def _generate_company_keywords(self, company_name: str, industry: str, industry_descriptions: Dict) -> List[str]:
        """Генерирует ключевые слова для компании"""
        keywords = set()
        
        # Основное название и варианты
        keywords.add(company_name.lower())
        
        # Разбиваем составные названия
        words = company_name.lower().split()
        keywords.update(words)
        
        # Добавляем отраслевые ключевые слова
        if industry in industry_descriptions:
            keywords.update(industry_descriptions[industry])
        
        # Специфические ключевые слова для известных компаний
        specific_keywords = {
            'сбербанк': ['сбер', 'банк', 'финансы', 'кредит'],
            'газпром': ['газ', 'трубопровод', 'голубое топливо', 'сила сибири'],
            'лукойл': ['нефть', 'бензин', 'заправка', 'топливо'],
            'норильский никель': ['никель', 'медь', 'металл', 'палладий'],
            'роснефть': ['нефть', 'нефтедобыча', 'скважина'],
            'аэрофлот': ['авиа', 'перевозки', 'рейс', 'аэропорт'],
        }
        
        if company_name in specific_keywords:
            keywords.update(specific_keywords[company_name])
        
        return list(keywords)
    
    def _generate_aliases(self, company_name: str) -> List[str]:
        """Генерирует альтернативные названия компании"""
        aliases = set()
        
        # Основное название
        aliases.add(company_name.lower())
        
        # Английские варианты
        english_mapping = {
            'газпром': 'gazprom',
            'лукойл': 'lukoil', 
            'сбербанк': 'sberbank',
            'роснефть': 'rosneft',
            'норильский никель': 'nornickel',
            'аэрофлот': 'aeroflot',
            'алроса': 'alrosa',
            'татнефть': 'tatneft'
        }
        
        if company_name in english_mapping:
            aliases.add(english_mapping[company_name])
        
        # Сокращения
        abbreviation_mapping = {
            'сбербанк': 'сбер',
            'московская биржа': 'московск биржа'
        }
        
        if company_name in abbreviation_mapping:
            aliases.add(abbreviation_mapping[company_name])
        
        return list(aliases)
    
    def _vectorize_knowledge_base(self):
        """Векторизует базу знаний для семантического поиска"""
        # Создаем текстовые описания для каждой компании
        company_descriptions = []
        self.company_tickers = []
        
        for ticker, info in self.company_knowledge_base.items():
            description = f"{info['company_name']} {info['industry']} {' '.join(info['keywords'])}"
            company_descriptions.append(description)
            self.company_tickers.append(ticker)
        
        # Векторизуем описания
        self.company_embeddings = self.sentence_model.encode(company_descriptions)
    
    def retrieve_relevant_companies(self, news_text: str, top_k: int = 5) -> List[Tuple[str, float]]:
        """Извлекает релевантные компании с помощью семантического поиска"""
        # Векторизуем новость
        news_embedding = self.sentence_model.encode([news_text])
        
        # Вычисляем косинусное сходство
        similarities = cosine_similarity(news_embedding, self.company_embeddings)[0]
        
        # Сортируем по релевантности
        relevant_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in relevant_indices:
            if similarities[idx] > 0.3:  # Порог релевантности
                results.append((self.company_tickers[idx], similarities[idx]))
        
        return results
    
    def extract_keywords_with_yake(self, text: str) -> List[str]:
        """Извлекает ключевые слова из текста"""
        keywords = self.kw_extractor.extract_keywords(text)
        return [kw for score, kw in keywords if score < 0.5]
    
    def extract_with_ner(self, text: str) -> List[str]:
        """Извлекает компании с помощью NER"""
        if not self.nlp:
            return []
            
        doc = self.nlp(text)
        organizations = [ent.text.lower() for ent in doc.ents if ent.label_ == 'ORG']
        
        found_tickers = []
        for org in organizations:
            for ticker, info in self.company_knowledge_base.items():
                if any(alias in org for alias in info['aliases']):
                    found_tickers.append(ticker)
        
        return list(set(found_tickers))
    
    def extract_direct_tickers(self, text: str) -> List[str]:
        """Ищет прямые упоминания тикеров"""
        ticker_pattern = r'\b([A-Z]{3,5})\b'
        matches = re.findall(ticker_pattern, text)
        
        valid_tickers = []
        for match in matches:
            if match in self.ticker_mapping.values():
                valid_tickers.append(match)
        
        return valid_tickers
    
    def is_financial_context(self, text: str, company_keywords: List[str]) -> bool:
        """Проверяет финансовый контекст"""
        financial_terms = [
            'акци', 'дивиденд', 'котировк', 'курс', 'цена', 'прибыль', 'убыток',
            'выручк', 'доход', 'отчетност', 'квартал', 'рекомендац', 'аналитик',
            'инвест', 'портфель', 'рынок', 'бирж', 'сделка', 'покупк', 'продаж'
        ]
        
        text_lower = text.lower()
        
        # Проверяем наличие финансовых терминов рядом с упоминаниями компании
        for keyword in company_keywords:
            if keyword in text_lower:
                # Ищем финансовые термины в окрестности ключевого слова
                words = text_lower.split()
                if keyword in words:
                    idx = words.index(keyword)
                    start, end = max(0, idx-3), min(len(words), idx+4)
                    context = ' '.join(words[start:end])
                    
                    if any(term in context for term in financial_terms):
                        return True
        
        return False
    
    def extract_tickers_advanced(self, news_text: str) -> List[str]:
        """Основной метод извлечения тикеров с использованием RAG"""
        if not isinstance(news_text, str) or not news_text.strip():
            return []
        
        found_tickers = set()
        
        # 1. Семантический поиск (RAG)
        relevant_companies = self.retrieve_relevant_companies(news_text, top_k=5)
        for ticker, score in relevant_companies:
            if score > 0.4:  # Достаточно высокая релевантность
                found_tickers.add(ticker)
        
        # 2. Прямое извлечение тикеров
        direct_tickers = self.extract_direct_tickers(news_text)
        found_tickers.update(direct_tickers)
        
        # 3. NER извлечение
        ner_tickers = self.extract_with_ner(news_text)
        found_tickers.update(ner_tickers)
        
        # 4. Ключевые слова и точное совпадение
        yake_keywords = self.extract_keywords_with_yake(news_text)
        news_lower = news_text.lower()
        
        for ticker, info in self.company_knowledge_base.items():
            # Проверяем точные совпадения с алиасами
            for alias in info['aliases']:
                if alias in news_lower:
                    if self.is_financial_context(news_text, [alias]):
                        found_tickers.add(ticker)
            
            # Проверяем совпадения ключевых слов
            for keyword in info['keywords']:
                if keyword in news_lower and len(keyword) > 3:  # Игнорируем короткие слова
                    if self.is_financial_context(news_text, [keyword]):
                        found_tickers.add(ticker)
        
        # 5. Фильтрация и ранжирование
        final_tickers = self._rank_and_filter_tickers(list(found_tickers), news_text)
        
        return final_tickers
    
    def _rank_and_filter_tickers(self, tickers: List[str], news_text: str) -> List[str]:
        """Ранжирует и фильтрует тикеры по релевантности"""
        if not tickers:
            return []
        
        # Вычисляем релевантность для каждого тикера
        scores = {}
        news_lower = news_text.lower()
        
        for ticker in tickers:
            score = 0
            info = self.company_knowledge_base[ticker]
            
            # Очки за точные совпадения
            for alias in info['aliases']:
                score += news_lower.count(alias) * 2
            
            # Очки за ключевые слова
            for keyword in info['keywords']:
                if keyword in news_lower:
                    score += 1
            
            # Очки за прямое упоминание тикера
            if ticker in news_text:
                score += 3
            
            scores[ticker] = score
        
        # Сортируем по убыванию релевантности
        sorted_tickers = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        
        # Возвращаем топ тикеры с достаточной релевантностью
        return [ticker for ticker, score in sorted_tickers if score >= 2][:5]  # Не более 5 тикеров

# Функциональный интерфейс (без классов)
def create_ticker_extractor(ticker_mapping: Dict[str, str]):
    """Создает экстрактор тикеров с заданным mapping"""
    rag_system = TickerRAGSystem(ticker_mapping)
    return rag_system.extract_tickers_advanced

def apply_to_dataframe(df, text_column: str, ticker_mapping: Dict[str, str], 
                      output_column: str = 'tickers') -> pd.DataFrame:
    """Применяет извлечение тикеров ко всему DataFrame"""
    extractor = create_ticker_extractor(ticker_mapping)
    
    df[output_column] = df[text_column].apply(extractor)
    return df

ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
if __name__ == "__main__":
    # Любые новые тикеры
    custom_ticker_mapping = {
        'аэрофлот': 'AFLT',
        'алроса': 'ALRS', 
        'газпром': 'GAZP',
        'лукойл': 'LKOH',
        'сбербанк': 'SBER',
        'роснефть': 'ROSN',
        'норильский никель': 'GMKN',
        'московская биржа': 'MOEX',
        'втб': 'VTBR',
        'яндекс': 'YNDX',  # Новая компания!
        'мвидео': 'MVID',   # Новая компания!
    }
    
    # Создаем экстрактор
    extract_tickers = create_ticker_extractor(custom_ticker_mapping)
    
    # Тестируем
    test_news = [
        "Акции Яндекса показали рекордный рост после выхода отчета",
        "М.Видео открывает новые магазины в регионах России",
        "Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов"
    ]
    
    print("Тестирование RAG-системы:")
    print("=" * 50)
    
    for news in test_news:
        tickers = extract_tickers(news)
        print(f"Новость: {news[:80]}...")
        print(f"Тикеры: {tickers}")
        print("-" * 50)

In [32]:
news_train_tickers.head(2)

,Unnamed: 0,publish_date,title,publication,tickers
0,25389,2020-01-01 14:00:00,Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов,"Тенденции в отрасли. Ключевые российские нефтегазовые компании смотрятся выгодно относительно аналогов с развитых и развивающихся рынков как на операционном, так и на финансовом уровнях. Можно отметить и комфортную долговую нагрузку, высокую рентабельность, стабильный денежный поток, высокие дивиденды. Капзатраты сектора высокие, однако большая часть приходится на масштабную инвестпрограмму ""Газпрома"", направленную в т.ч. на развитие ключевого проекта ""Сила Сибири"". 2019 год сложился неблагоприятно с точки зрения долларовых цен на нефть и газ, однако компании смогли заработать на девальвации рубля и экономии на налогах за счет измененного демпфирующего механизма. Отдельно отметим сохраняющийся высокий уровень дивидендных выплат в отрасли. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Выручка, млрд долл. Ключевые события. Введение обратного акциза на нефть (демпфирующий механизм) в начале года было сопряжено со сложностями адаптации к рыночным реалиям, однако корректировка позволила компаниям заработать. Переход к новой дивидендной политике – ""Газпром"" (50% от прибыли по МСФО в 2021 г.) и ""ЛУКОЙЛ"" (приоритет дивидендам, а не выкупу своих акций). Мультипликаторы Источник: Bloomberg: ПСБ Аналитика & Стратегия Риски. Основные риски отрасли сопряжены с возможным дальнейшим ухудшением ценовой конъюнктуры и изменениями в сфере налогообложения. Средние значения денежных потоков, млн долл. Источник: Bloomberg: ПСБ Аналитика & Стратегия Рекомендации: ""Газпром"". - Согласно новой дивидендной политике, ""Газпром"" планирует в течение 3 лет довести коэффициент дивидендных выплат до 50%, благодаря чему дивидендная доходность может превысить 10%. - Комфортная долговая нагрузка, которая не будет препятствовать уменьшению дивидендных выплат. - Достаточный для выплат дивидендов акционерам объем FCF и отсутствие дальнейшего наращивания инвестпрограммы. Динамика цен на нефть и газ, долл./мБТЕ Источник: Bloomberg, ПСБ Аналитика & Стратегия",[]
1,25388,2020-01-02 15:00:00,ММК выгодно отличает высокая экспозиция на внутренний рынок,"Тенденции в отрасли. Ключевые российские сталелитейные компании сохраняют конкурентоспособность на операционном и финансовом уровне относительно аналогов. Высокая рентабельность сочетается с низкой долговой нагрузкой, стабильными денежными потоками и высокой дивидендной доходностью. Прошедший год был неблагоприятным с точки зрения ценовой конъюнктуры, но высокая премия на внутреннем рынке сыграла в плюс, поэтому существенно просел экспорт. Учитывая циклический характер развития отрасли и замедление промышленного производства, финансовые результаты остались на уровне прошлого года или ухудшились, что оказало давление на котировки. Положение относительно конкурентов Источник: Bloomberg: ПСБ Аналитика & Стратегия Размер шара – Funds form Operation, млрд долл. Корпоративные события. Из отраслевых событий отметим введение пошлин на импорт г/к проката, а также запрет на экспорт лома. В период до 2023г. рост спроса на металлургическую продукцию должна обеспечить реализация национальных проектов, которые позволят увеличить ежегодный выпуск стали на 4%. Угрозу дивидендам не несут инвестпрограммы компаний, которые остались без изменения. Из корпоративных событий центральной темой года стала сделка по продажа американского подразделения ТМК. Мультипликаторы Источник: Bloomberg Риски. Условия на рынке сырья складываются неплохо, но сырьевой сегмент уже не способен перекрывать по рентабельности стальной. Внешние условия могут негативно повлиять на FCF и скажутся на снижении дивидендных выплат. Средние значения денежных потоков, млн долл. Источник: ММК, НЛМК, ""Северсталь"", Evraz, ПСБ Аналитика&Стратегия Рекомендации: ММК. - Долговая нагрузка самая низкая среди российских металлургов. Объем денежных средств полностью

In [26]:
# print(news_train['tickers'].unique().tolist())
# news_train['tickers']